In [1]:
from bs4 import BeautifulSoup
from selenium import webdriver
import pymongo
# import synonyms
import time
import random

ops = webdriver.ChromeOptions()
ops.add_argument('--headless')
ops.add_argument('--no-sandbox')
driver = webdriver.Chrome(options=ops)
# driver.implicitly_wait(5)

In [2]:
URL = 'https://wh.58.com/shangpucz/pn'
client = pymongo.MongoClient("mongodb://101.132.171.223:27017/")
db = client['shoprec']
col = db['house']

In [3]:
PAGE_NUM = 70
def getHouse(idx):
    driver = webdriver.Chrome(options=ops)
    url = URL + str(idx)
    print(url)
    driver.get(url)
    soup = BeautifulSoup(driver.page_source, 'html.parser')
    body = soup.body
    ul = body.find('ul', attrs={'class': 'house-list-wrap'})
    lis = ul.find_all('li')
    return list(map(getInfo, lis))
    
def getInfo(li):
    info = li.contents[3]
    price_info = li.contents[7]
    title = info.find('span', attrs={'class': 'title_des'}).text
    price = price_info.find('b').text
    baseInfo = info.find_all('p', attrs={'class': 'baseinfo'})
    _type = ""
    if len(baseInfo[0]) == 7:       
        area = baseInfo[0].contents[1].text
        _type = baseInfo[0].contents[3].text
        status = baseInfo[0].contents[5].text
    elif len(baseInfo[0]) == 5:
        area = baseInfo[0].contents[1].text
        status = baseInfo[0].contents[3].text  
    location = baseInfo[1].contents[1].text
    address = ""
    if len(baseInfo[1]) == 7:
        address = baseInfo[1].contents[3].text
    tags_info = info.find('p', attrs={'class': 'tag-wrap'})
    tags = []
    if tags_info != None:
        tags = tags_info.find_all('span')
        tags = list(map(lambda x: x.text, tags))
    return {
        'title': title,
        'price': price,
        'area': area,
        'type': _type,
        'location': location,
        'address': address,
        'status': status,
        'tags': tags
    }

In [ ]:
for i in range(1, PAGE_NUM + 1):
    data = getHouse(i)
    col.insert_many(data)
    print("Insertting...")
    time.sleep(random.randint(1, 5))

https://wh.58.com/shangpucz/pn1
Insertting...
https://wh.58.com/shangpucz/pn2
Insertting...
https://wh.58.com/shangpucz/pn3
